# Assignment: Preparing Data for Analysis (Modified Titanic)

![](https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/%20files/hw.png)

In [304]:
#Import the libraries
import numpy as np
import pandas as pd

In [305]:
df = pd.read_csv('https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/datasets/hw/titanic_training_dataset_v2.csv')

In [306]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0,PC 17599,71.2833,C85,C
2,3,1.0,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0,113803,53.1000,C123,S
4,5,0.0,3.0,"Allen, Mr. William Henry",male,35.0,0.0,0,373450,8.0500,NaN,S


In [307]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     865 non-null    float64
 2   Pclass       828 non-null    float64
 3   Name         866 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        858 non-null    float64
 7   Parch        891 non-null    int64  
 8   Ticket       852 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     803 non-null    object 
dtypes: float64(5), int64(2), object(5)
memory usage: 83.7+ KB


# 1) Load data & review the data

<font color='blue'>Q1) What is the shape of this dataset? (rows & columns)</font>

In [308]:
df.shape

(891, 12)

# 2) Drop unqualified variables

*   Drop variables with missing > 50%
*   Drop categorical variables with flat values > 70% (variables with the same value in the same column)  
<font color='red'>*change to all flat values > 70%*</font>  
<font color='blue'>Q2) How many columns do we have left?</font>




In [309]:
# Drop columns with more than 50% missing values
half_count = len(df) / 2
df = df.dropna(thresh=half_count,axis=1)
print(df.shape)

(891, 11)


In [310]:
# check column with same flat values more than 70% of data row
seventy_count = len(df) / 10 * 7
target_col = []
for col in df.columns:
    if ((df[col].value_counts()>seventy_count).sum()):
        print(df[col].value_counts())
        print()
        target_col.append(col)

Parch
0    678
1    118
2     80
5      5
3      5
4      4
6      1
Name: count, dtype: int64



In [311]:
# Drop column with same flat values more than 70% of data row
df = df.drop(target_col,axis=1)
print(df.shape)

(891, 10)


# 3) Remove all rows with missing target (the variable "Survived")

<font color='blue'>Q3) How many rows do we have left?</font>

In [312]:
print(df.shape)
df = df.dropna(subset=['Survived'])
print(df.shape)

(891, 10)
(865, 10)


# 4) Handle outliers

For the variable “Fare”, replace outlier values with the boundary values


*   If value < (Q1 - 1.5IQR), relace with (Q1 - 1.5IQR)
*   If value > (Q3 + 1.5IQR), relace with (Q3 + 1.5IQR)

<font color='blue'>Q4) What is the mean of “Fare” after replacing the outliers (round 2 decimal points)?</font>



In [313]:
q75, q25 = np.percentile(df['Fare'], [75 ,25])
iqr = q75 - q25

min = q25 - (iqr*1.5)
max = q75 + (iqr*1.5)
print('Box range:', min,max)
df_no_outlier = df.copy()
print('Number of outliers before:',len(df_no_outlier[(df_no_outlier['Fare']<min) | (df_no_outlier['Fare']>max)]['Fare']))
df_no_outlier.loc[df_no_outlier['Fare']<min, 'Fare'] = min
df_no_outlier.loc[df_no_outlier['Fare']>max, 'Fare'] = max
print('Number of outliers after:',len(df_no_outlier[(df_no_outlier['Fare']<min) | (df_no_outlier['Fare']>max)]['Fare']))

Box range: -26.7605 65.6563
Number of outliers before: 113
Number of outliers after: 0


In [314]:
print('Mean before:', round(df['Fare'].mean(),2))
print('Mean after:', round(df_no_outlier['Fare'].mean(),2))

Mean before: 32.39
Mean after: 24.04


# 5) Impute missing value



*   Show the number of missing values in each variable
*   Impute missing values with mean & mode
*   Show the number of missing values again after missing value imputation

<font color='blue'>Q5) Which variable has the largest number of missing values?</font>



In [315]:
null_counts = df_no_outlier.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))
filtered_df = df_no_outlier.copy()

Number of null values in each column:
PassengerId      0
Survived         0
Pclass          63
Name            25
Sex              0
Age            172
SibSp           33
Ticket          39
Fare             0
Embarked        87
dtype: int64


In [316]:
from sklearn.impute import SimpleImputer
num_cols = filtered_df.select_dtypes(include=np.number).columns
filtered_df_num = filtered_df[num_cols]

num_imp=SimpleImputer(missing_values=np.NaN, strategy='mean')

# filtered_df[num_cols] = num_imp.fit_transform(filtered_df_num)
filtered_df[num_cols] = pd.DataFrame(num_imp.fit_transform(filtered_df_num), index=filtered_df_num.index)
print(filtered_df.isnull().sum())

PassengerId     0
Survived        0
Pclass          0
Name           25
Sex             0
Age             0
SibSp           0
Ticket         39
Fare            0
Embarked       87
dtype: int64


In [317]:
# Index test
# temp = pd.DataFrame({
#     'col1': ['a', 'a', 'b', 'b', 'a'],
#     'col2': [1, 2, 3, np.nan, 5],
#     'col3': [1, 2, 3, 4, 5]
# })
# # print(temp)
# temp.dropna(subset=['col2'], inplace=True)
# print(temp)

# changing_col23 = pd.DataFrame([[5,5],[4,4],[3,3],[2,2]])
# # changing_col23 = [[5,5],[4,4],[3,3],[2,2]]
# temp2 = temp.copy()
# temp2[['col2','col3']] = changing_col23
# print(temp2)

In [318]:
cat_cols = filtered_df.select_dtypes(exclude=np.number).columns
filtered_df_cat = filtered_df[cat_cols]

cat_imp=SimpleImputer(missing_values=np.NaN, strategy='most_frequent')
filtered_df[cat_cols] = pd.DataFrame(cat_imp.fit_transform(filtered_df_cat), index=filtered_df_num.index)

In [319]:
null_counts = filtered_df.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

Number of null values in each column:
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


# 6) Convert categorical to numeric values



*   Drop the variables “Name” & “Ticket”
*   For the variables “Sex” & “Embarked”, perform the dummy coding and drop the first level. Also, drop those original variables (“Sex” & “Embarked”)

<font color='blue'>Q6) How many columns do we have?</font>



In [320]:
drop_cols = ['Name', 'Ticket']
filtered_df.drop(drop_cols, axis=1, inplace=True)

In [321]:
# filtered_df.columns

In [322]:
nominal_columns = ["Sex", "Embarked"]
dummy_df = pd.get_dummies(filtered_df[nominal_columns], drop_first=True)
filtered_df = pd.concat([filtered_df, dummy_df], axis=1)
filtered_df = filtered_df.drop(nominal_columns, axis=1)
print(filtered_df.shape)

(865, 9)


In [323]:
filtered_df.head()

,PassengerId,Survived,Pclass,Age,SibSp,Fare,Sex_male,Embarked_Q,Embarked_S
0,1.0,0.0,3.0,22.0,1.0,7.2500,True,False,True
1,2.0,1.0,1.0,38.0,1.0,65.6563,False,False,False
2,3.0,1.0,3.0,26.0,0.0,7.9250,False,False,True
3,4.0,1.0,1.0,35.0,1.0,53.1000,False,False,True
4,5.0,0.0,3.0,35.0,0.0,8.0500,True,False,True


# 7) Partition data



*   Split train/test split with stratification using 70%:30% and random seed with 12345
*   Show a proportion between survived (1) and died (0) in all data sets (total data, train, test)

<font color='blue'>Q7) What is a proportion between survived (1) and died (0) in the training data?</font>



In [324]:
from sklearn.model_selection import train_test_split

y = filtered_df.pop('Survived')
X = filtered_df

X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,test_size=0.30, random_state=12345)

In [325]:
X_train.shape, X_test.shape

((605, 8), (260, 8))